In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import time
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import random
import csv
random.seed(1991)
import fastparquet
import spacy
import fasttext
import glob, os
import re
from os.path import isfile
done_files=[]
os.chdir("/data/dharp/compounds/datasets/google/")
for file in glob.glob("*.pkl"):
    done_files.append(file.split('.')[0])


fivegram_list='0 1 2 3 4 5 6 7 8 9 a_ aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az b_ ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz c_ ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz d_ da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz e_ ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez f_ fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz g_ ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz h_ ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz i_ ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz k_ ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz l_ la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz m_ ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq mr ms mt mu mv mw mx my mz n_ na nb nc nd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt nu nv nw nx ny nz o_ oa ob oc od oe of og oh oi oj ok ol om on oo op oq or os ot other ou ov ow ox oy oz p_ pa pb pc pd pe pf pg ph pi pj pk pl pm pn po pp pq pr ps pt pu punctuation pv pw px py pz q_ qa qb qc qd qe qf qg qh qi qj ql qm qn qo qp qq qr qs qt qu qv qw qx qy qz r_ ra rb rc rd re rf rg rh ri rj rk rl rm rn ro rp rq rr rs rt ru rv rw rx ry rz s_ sa sb sc sd se sf sg sh si sj sk sl sm sn so sp sq sr ss st su sv sw sx sy sz t_ ta tb tc td te tf tg th ti tj tk tl tm tn to tp tq tr ts tt tu tv tw tx ty tz u_ ua ub uc ud ue uf ug uh ui uj uk ul um un uo up uq ur us ut uu uv uw ux uy uz v_ va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz w_ wa wb wc wd we wf wg wh wi wj wk wl wm wn wo wp wq wr ws wt wu wv ww wx wy wz x_ xa xb xc xd xe xf xg xh xi xj xk xl xm xn xo xp xq xr xs xt xu xv xw xx xy xz y_ ya yb yc yd ye yf yg yh yi yj yk yl ym yn yo yp yq yr ys yt yu yv yw yx yy yz z_ za zb zc zd ze zf zg zh zi zj zk zl zm zn zo zp zq zr zs zt zu zv zw zx zy zz'
large_files=['an','of','to','in','ad','wh','be','ha','is','co','wa','he','no','it','wi','fo','re','as','on','we','punctuation','th','ma','pr','ar','ip','sh','ca','so','hi','bu','al','se','de','by','wo','st','fr','di','mo','su','at','or','yo','me','li','pa','do','ex','le','pe','po','if','ne','fi','un','fa','sa','ch','la','lo','ac','ho','mu','go','si','en','ev','tr']

fivegram_list=fivegram_list.split(" ")
random.shuffle(fivegram_list)
small_files=list(set(fivegram_list).difference(large_files))

fmodel = fasttext.load_model('/data/dharp/packages/lid.176.bin')
#nlp = spacy.load('en_core_web_sm')


nn='(?!NOUN).*'
comp='NOUN\sNOUN'
word='.*'

n1=f'^{comp}\s{nn}\s{comp}$'
n2=f'^{comp}\s{nn}\s{word}\s{word}$'
n3=f'^{nn}\s{comp}\s{nn}\s{word}$'
n4=f'^{word}\s{nn}\s{comp}\s{nn}$'
n5=f'^{word}\s{word}\s{nn}\s{comp}$'


def delist_lang(lst):
    lang_lst=[]
    for i,lang in enumerate(lst):
        if not lang:
            lang_lst.append(None)
        else:
            lang_lst.append(lang[0])
    return lang_lst


def significance(lst):
    significance_list=[]
    for l in lst:
        if len(l)>1:
            significance_list.append(abs(l[0]-l[1])/np.mean(l[0]+l[1])>0.1)
            #print(f'{conf[0]} {conf[1]} {abs(conf[0]-conf[1])/np.mean(conf[0]+conf[1])>0.1}')
        else:
            significance_list.append(True)
    return significance_list

div_lsts=np.array_split(small_files, 5)

def chunked_dataset_extracter(df):
    df.columns=['fivegram_pos','year','count']
    
    df=df.loc[df.fivegram_pos.str.match("^"+keep_string*5+"$",na=False)]
    df=df.loc[~(df.fivegram_pos.str.match("^"+remove_string*5+"$",na=False))]
    df.fivegram_pos=df.fivegram_pos.str.lower()

    df=df.groupby(['fivegram_pos','year'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    return df


def dataset_extracter(letter):
    CHUNKSIZE = 100_000_000
    #print(f"\nStarted with letter(s) {letter}")
    cur_time=time.time()
    total_df_shape=0
    df_list=[]
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2],chunksize=CHUNKSIZE)    
    for df in dfs:
        total_df_shape+=df.shape[0]
        df_list.append(chunked_dataset_extracter(df))
    complete_df=pd.concat(df_list,ignore_index=True,sort=False)
    
    after_shape=complete_df.shape[0]

    if after_shape!=0:
        complete_df.to_pickle('/data/dharp/compounds/datasets/google/'+letter+'.pkl')
        print(f"Finished with letter(s) {letter} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
        print(f"Letter(s) {letter} took time {(time.time()-cur_time):0.2f} seconds")
        print("\n")
    else:
        print(f"Finished with letter(s) {letter} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
        print(f"Letter(s) {letter} took time {(time.time()-cur_time):0.2f} seconds")
        print("\n")

def dataset_downloader_whole(cur_list):
    n_proc = 60

    pool = Pool(n_proc)
    pool.imap_unordered(dataset_extracter,cur_list,chunksize=1) 
    pool.close()
    pool.join()
    
    
def large_df_processor(letter):
    CHUNKSIZE = 200_000_000
    num_partitions = 100
    total_df_shape=0
    df_list=[]
    cur_time=time.time()
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2],chunksize=CHUNKSIZE)    
    print(len(dfs))
    for i,df in enumerate(dfs):
        print(f'Split num {i+1}')
        total_df_shape+=df.shape[0]
        df_split = np.array_split(df, num_partitions)
        pool = Pool(num_partitions)
        print('Started parallelization')
        results=pool.map_async(chunked_dataset_extracter,df_split)
        pool.close()
        pool.join()
        curr_df_list=results.get()
        #df_list.extend(curr_df_list)
        print(f'Total time taken for split num {i+1}: {round(time.time()-cur_time)} secs')
    #print(df_list)
    
    
    
    total_df_shape=df.shape[0]
    after_shape=complete_df.shape[0]
    
    
    complete_df.to_pickle('/data/dharp/compounds/datasets/google/'+letter+'.pkl')
    print(f"Finished with letter(s) {letter} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f"Letter(s) {letter} took time {(time.time()-cur_time):0.2f} seconds")
    print("\n")

In [7]:
" ".join(small_files)

'ds xn sq up ed ce tv zs cr zv ka wp jz vs jd jn sk hw sg kh nv oc lq ro vj dc am oh dw tu gj tq tw rx nt cd ug zh 0 h_ 7 ii vl el vo hm ja os xv cc dl fc qr pf cx je sc dh et sw vq ms yv nb ud sv jo fb nc jm nu xp xe ps dr nx kv gi pd sz xd ov ub o_ mj 6 fd qe ux pz rd wx other qh iu rp iy ek lv xy ax vu oy cg og av kd id wv ie mg um rf eg bf fl rt wn pp qx td vz ae y_ n_ qc kq bo yx cs gd yt ig lw sy wj wk ix ik em dg bm pj fw lr c_ nn qw pq pc tj qp qv vf fs mx tn ia ak es dm dk gp kg jl lx xc zy s_ ym eh jk kw im hg lm xx qi g_ rh kr cz oe nw mb zd nh pi ta gt du vg oo tb mh ij ly ox rm ge ee ep rg qa hj ib ue ic 8 wf km ew mk uo hp om jc nq tt nz mp vc gn pt ur gc pk rc xg vp da jv dt ni kc bg vn ph nf hr qb gb zl bn zu cv pw zo cl my tc wy mc qg xr qd lu js cw lk ea q_ kl db z_ gv hx gq dv qf ll dy jg nl xl ks uc kx qz fy d_ jt ej ff zx bv yj hb tl ec zc yz jx bd hu tg px yn fe gf kj gr zz ya dx rb aw ot yp mq yu ty bq cp yw mt oa qu br ut ga cb iz ap vh uv ey t_ e_ ih ou uz zp 5

In [2]:
def sent_maker(sent_lst):
    ret_sents=[]
    ret_pos=[]
    for sent in sent_lst:
        cur_words=[]
        cur_pos=[]
        for word_pos in sent.split(' '):
            word,*pos=word_pos.rsplit('_',1)
            if pos==[]:
                pos=''
            else:
                #print(pos)
                pos=pos[0]
            cur_words.append(word)
            #print(word)
            #print(pos)
            cur_pos.append(pos)
            cur_sent=' '.join(cur_words)
            pos_sent=' '.join(cur_pos)
            cur_sent=re.sub('_', '', cur_sent)
        ret_sents.append(cur_sent)
        ret_pos.append(pos_sent)
    return ret_sents,ret_pos

In [3]:
len(small_files)

644

In [4]:
def ner_lemma_reducer(sent):
    ner_sent=[]
    lemma=[]
    pos=[]
    parse=[]
    ner=[]
    parsed_sent=nlp(sent)
    for token in parsed_sent:
        parse.append(token.text)
        lemma.append(token.lemma_)
        pos.append(token.pos_)

    #print(parse)
    parse_sent=' '.join(parse)
    lemma_sent=' '.join(lemma)
    pos_sent=' '.join(pos)
    if parsed_sent.ents:
        for ent in parsed_sent.ents:
            cur_ner='_'.join([str(ent.start_char), str(ent.end_char), ent.label_])
            ner.append(cur_ner)
    else:
        ner.append('')
    ner_sent=' '.join(ner)
    
    return parse_sent,ner_sent,lemma_sent,pos_sent

In [5]:
def lang_tagger(parsed_sent):
    labels,confs=fmodel.predict(parsed_sent,k=-1,threshold=0.1)
    lang_list=delist_lang(labels)    
    significance_list=significance(confs)
    assert len(lang_list)==len(significance_list)
    return lang_list,significance_list

In [6]:
def small_df_processor(letter):
    cur_time=time.time()
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    df   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2])    
    df.columns=['fivegram_pos','year','count']
    df=df.loc[df.year>=1800]
    total_df_shape=df.shape[0]
    index_df=df.groupby(['fivegram_pos'])['count'].sum().reset_index()
    index_df.columns=['old_index','total_count']

    index_df=trial(index_df)
    ntypes=index_df.shape[0]
    ntokens=index_df.total_count.sum()

    types_perc=round(ntypes/df.shape[0]*100,3)
    print(f'Number of types: {ntypes}, perc. of unique types (decade agnostic): {types_perc}%')

    print(f'Number of tokens: {ntokens}, ratio of tokens to types: {round(ntokens/ntypes,3)}')

    ncomptypes=np.sum(index_df.comp_class!=0)
    ncomptypes_perc=round(ncomptypes/ntypes*100,3)
    print(f'Number of compounds types: {ncomptypes}, perc. of compound types: {ncomptypes_perc}%')

    comp_count=index_df.loc[index_df.comp_class!=0,'total_count'].sum()
    comp_count_perc=round(comp_count/ntokens*100,3)
    print(f'Compound count: {comp_count}, perc. of compound tokens: {comp_count_perc}%')

    words_df=index_df.loc[index_df.pos_sent.str.contains('NOUN')].reset_index(drop=True)
    words_df['nner']=words_df.ner_sent.str.count(' ').add(1)
    words_df['nX']=words_df.pos_sent.str.count('X')-words_df.pos_sent.str.count('AUX')
    words_df=words_df.loc[~(words_df.nX>=3)]
    words_df=words_df.loc[words_df.nner<2]        
        
    words=pd.merge(df,words_df,left_on='fivegram_pos',right_on='old_index',how='right')
    words=words.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
    words.reset_index(inplace=True)

    words.to_pickle(f'/data/dharp/compounds/datasets/google/{letter}.pkl')
    #phrases_df=words_df.loc[words_df.pos_sent.str.contains('NOUN NOUN')].reset_index(drop=True)
    #phrases=pd.merge(df,phrases_df,left_on='fivegram_pos',right_on='old_index',how='right')
    #phrases=phrases.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
    #phrases.reset_index(inplace=True)

    #comp_df=phrases_df.loc[phrases_df.comp_class!=0].reset_index(drop=True)
    #compounds=pd.merge(df,comp_df,left_on='fivegram_pos',right_on='old_index',how='right')
    #compounds=compounds.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
    #compounds.reset_index(inplace=True)

    print(f'Total time taken for letter {letter}: {round(time.time()-cur_time)} secs')
    with open(f'/data/dharp/compounds/datasets/stats/{letter}.txt','w') as f:
        f.write(f'{letter}\t1\t{ntypes}\t{ntokens}\t{ncomptypes}\t{comp_count}\n')
        
   

In [7]:
def trial(df):

    df['sent']=sent_maker(df.old_index)
    
    results=df.sent.apply(ner_lemma_reducer)

    results_df=pd.DataFrame.from_records(results)
    results_df.columns=['parse_sent','ner_sent','lemma_sent','pos_sent']

    index_df=pd.concat([df,results_df],axis=1)
    lang_list,significance_list=lang_tagger(index_df.parse_sent.values.tolist())
    
    index_df['lang']=lang_list
    index_df['lang_conf']=significance_list
    index_df.lang=index_df.lang.str.split('_',n=4).str[-1]
    index_df=index_df.loc[(index_df.lang=='en') &(index_df.lang_conf==True)]
    index_df['nwords']=index_df.pos_sent.str.count(' ').add(1)
    index_df=index_df.loc[index_df.nwords==5]
    index_df.lemma_sent=index_df.lemma_sent.str.lower()
    index_df.pos_sent=index_df.pos_sent.str.replace('PROPN','NOUN')
    index_df['comp_class']=0

    index_df.loc[index_df.pos_sent.str.contains(n1),'comp_class']=1
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n2),'comp_class']=2
    index_df.loc[index_df.pos_sent.str.contains(n3),'comp_class']=3
    index_df.loc[index_df.pos_sent.str.contains(n4),'comp_class']=4
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n5),'comp_class']=5

    index_df.drop(['parse_sent','sent','lang','lang_conf','nwords'],axis=1,inplace=True)
    return index_df

In [8]:
def dataset_downloader_whole(cur_list):
    n_proc = 100

    pool = Pool(n_proc)
    pool.imap_unordered(small_df_processor,cur_list,chunksize=1) 
    pool.close()
    pool.join()

In [23]:
    letter='co'
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    df   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2])    
    df.columns=['fivegram_pos','year','count']
    df=df.loc[df.year>=1800]
    total_df_shape=df.shape[0]
    index_df=df.groupby(['fivegram_pos'])['count'].sum().reset_index()
    index_df.columns=['old_index','total_count']

In [26]:
df=index_df.copy()
sents,pos=sent_maker(df.old_index)
df[['sent','orig_pos']]=pd.DataFrame({'sent':sents,'orig_pos':pos})

In [27]:
def ner_parser(df):
    return df.sent.apply(ner_lemma_reducer)

In [28]:
num_partitions=250
df_split = np.array_split(df, num_partitions)
pool = Pool(num_partitions)
print('Started parallelization')
results=pool.map_async(ner_parser,df_split)
pool.close()
pool.join()
curr_df_list=results.get()

Started parallelization


In [29]:
flat_curr_df_list = [item for sublist in curr_df_list for item in sublist]

In [30]:
results_df=pd.DataFrame.from_records(flat_curr_df_list)
results_df.columns=['parse_sent','ner_sent','lemma_sent','pos_sent']
index_df=pd.concat([df,results_df],axis=1)
index_df

,old_index,total_count,sent,orig_pos,parse_sent,ner_sent,lemma_sent,pos_sent
0,"C.O. ' s , and",82,"C.O. ' s , and",,"C.O. ' s , and",,"C.O. ' s , and",PROPN PART NOUN PUNCT CCONJ
1,C.O. ' s in the,104,C.O. ' s in the,,C.O. ' s in the,,C.O. ' s in the,PROPN PART NOUN ADP DET
2,C.O. ' s of the,122,C.O. ' s of the,,C.O. ' s of the,,C.O. ' s of the,PROPN PART NOUN ADP DET
3,"C.O. ' s office ,",88,"C.O. ' s office ,",,"C.O. ' s office ,",0_4_ORG,"C.O. ' s office ,",PROPN PART NOUN NOUN PUNCT
4,C.O. ' s office .,161,C.O. ' s office .,,C.O. ' s office .,0_4_ORG,C.O. ' s office .,PROPN PART NOUN NOUN PUNCT
...,...,...,...,...,...,...,...,...
23206726,côté_X à_X l'_X autre_X de_X,49,côté à l' autre de,X X X X X,côté à l ' autre de,0_9_PERSON 10_18_PERSON,côté à l ' autre de,PROPN PROPN PROPN PUNCT PROPN PROPN
23206727,"côtés de l' Atlantique ,",58,"côtés de l' Atlantique ,",,"côtés de l ' Atlantique ,",0_11_PERSON 12_22_GPE,"côtés de l ' Atlantique ,",PROPN X PROPN PUNCT PROPN PUNCT
23206728,"côtés à la fois ,",41,"côtés à la fois ,",,"côtés à la fois ,",0_15_PERSON,"côtés à la fois ,",PROPN PROPN PROPN PROPN PUNCT
23206729,"côtés_X de_X l'_X Atlantique_NOUN ,_.",58,"côtés de l' Atlantique ,",X X X NOUN .,"côtés de l ' Atlantique ,",0_11_PERSON 12_22_GPE,"côtés de l ' Atlantique ,",PROPN X PROPN PUNCT PROPN PUNCT


In [34]:
    lang_list,significance_list=lang_tagger(index_df.parse_sent.values.tolist())
    
    index_df['lang']=lang_list
    index_df['lang_conf']=significance_list
    index_df.lang=index_df.lang.str.split('_',n=4).str[-1]
    index_df=index_df.loc[(index_df.lang=='en') &(index_df.lang_conf==True)]
    index_df['nwords']=index_df.pos_sent.str.count(' ').add(1)
    #index_df=index_df.loc[index_df.nwords==5]
    index_df.lemma_sent=index_df.lemma_sent.str.lower()
    index_df.pos_sent=index_df.pos_sent.str.replace('PROPN','NOUN',regex=False)
    index_df.pos_sent=index_df.pos_sent.str.replace('.CONJ','CONJ',regex=True) ##TOFIX
    index_df.pos_sent=index_df.pos_sent.str.replace('AUX','VERB',regex=True)
    index_df.orig_pos=index_df.orig_pos.str.replace('.','PUNCT',regex=False)
    index_df['comp_class']=0

<ipython-input-34-d76efd71586d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_df['lang']=lang_list
<ipython-input-34-d76efd71586d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_df['lang_conf']=significance_list
/data/dharp/packages/miniconda3/lib/python3.8/site-packages/pandas/core/generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [35]:
unique_ids=index_df.sent.value_counts().to_frame().reset_index()

In [39]:
unique_ids

,index,sent
0,COUNT ( * ) FROM,10
1,Commander -- in -- Chief,9
2,Complete each of the following,9
3,commonly referred to as ',9
4,comments as follows : ',8
...,...,...
11503158,convinced that he is surrounded,1
11503159,"connected with it , first",1
11503160,"consisting of hundreds , nay",1
11503161,content ( see Chapter 5,1


In [38]:
unique_ids.sent.describe()

count   11503163.000
mean           1.989
std            0.183
min            1.000
25%            2.000
50%            2.000
75%            2.000
max           10.000
Name: sent, dtype: float64

In [43]:
index_df.orig_pos.value_counts(normalize=True)*100

                           50.267
NOUN ADP DET NOUN ADP       1.132
NOUN ADP DET NOUN PUNCT     1.039
NOUN ADP DET ADJ NOUN       0.763
VERB ADP DET NOUN ADP       0.723
                            ...  
NOUN X X PRON PUNCT         0.000
NOUN CONJ CONJ PUNCT DET    0.000
ADJ PRON DET NOUN ADP       0.000
ADV ADV PRT DET VERB        0.000
NOUN DET ADJ PRON NUM       0.000
Name: orig_pos, Length: 36863, dtype: float64

In [51]:
index_df.loc[index_df.orig_pos!=index_df.pos_sent]

,old_index,total_count,sent,orig_pos,parse_sent,ner_sent,lemma_sent,pos_sent,lang,lang_conf,nwords,comp_class
0,"C.O. ' s , and",82,"C.O. ' s , and",,"C.O. ' s , and",,"c.o. ' s , and",NOUN PART NOUN PUNCTCONJ,en,True,5,0
1,C.O. ' s in the,104,C.O. ' s in the,,C.O. ' s in the,,c.o. ' s in the,NOUN PART NOUN ADP DET,en,True,5,0
2,C.O. ' s of the,122,C.O. ' s of the,,C.O. ' s of the,,c.o. ' s of the,NOUN PART NOUN ADP DET,en,True,5,0
3,"C.O. ' s office ,",88,"C.O. ' s office ,",,"C.O. ' s office ,",0_4_ORG,"c.o. ' s office ,",NOUN PART NOUN NOUN PUNCT,en,True,5,0
4,C.O. ' s office .,161,C.O. ' s office .,,C.O. ' s office .,0_4_ORG,c.o. ' s office .,NOUN PART NOUN NOUN PUNCT,en,True,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23206596,cozying up to him .,60,cozying up to him .,,cozying up to him .,,cozye up to -pron- .,VERB ADP ADP PRON PUNCT,en,True,5,0
23206597,cozying_VERB up_PRT to_PRT each_DET other_ADJ,67,cozying up to each other,VERB PRT PRT DET ADJ,cozying up to each other,,cozye up to each other,VERB ADP ADP DET ADJ,en,True,5,0
23206598,cozying_VERB up_PRT to_PRT him_PRON ._.,59,cozying up to him .,VERB PRT PRT PRON PUNCT,cozying up to him .,,cozye up to -pron- .,VERB ADP ADP PRON PUNCT,en,True,5,0
23206599,"cozzening , hidden lord and",70,"cozzening , hidden lord and",,"cozzening , hidden lord and",,"cozzening , hide lord and",NOUN PUNCT VERB NOUNCONJ,en,True,5,0


In [52]:
(index_df.ner_sent.value_counts(normalize=True)*100).head(30)

                 82.819
0_8_ORG           0.753
0_7_ORG           0.539
0_9_ORG           0.248
0_11_ORG          0.236
0_6_ORG           0.225
0_4_ORG           0.143
0_10_ORG          0.135
0_12_ORG          0.132
0_16_ORG          0.109
0_5_ORG           0.106
0_14_ORG          0.102
0_20_ORG          0.101
0_15_ORG          0.099
12_15_CARDINAL    0.098
0_19_ORG          0.091
0_17_ORG          0.086
0_18_ORG          0.086
0_13_ORG          0.083
0_21_ORG          0.077
0_22_ORG          0.077
13_16_CARDINAL    0.077
13_17_DATE        0.076
0_23_ORG          0.073
14_17_CARDINAL    0.071
0_25_ORG          0.065
0_24_ORG          0.063
15_18_CARDINAL    0.060
0_26_ORG          0.059
0_27_ORG          0.058
Name: ner_sent, dtype: float64

In [57]:
index_df.loc[index_df.ner_sent.str.contains('ORG')].sample(50)

,old_index,total_count,sent,orig_pos,parse_sent,ner_sent,lemma_sent,pos_sent,lang,lang_conf,nwords,comp_class
1551390,"Congress_NOUN had_VERB ,_. in_ADP a_DET",52,"Congress had , in a",NOUN VERB PUNCT ADP DET,"Congress had , in a",0_8_ORG,"congress have , in a",NOUN VERB PUNCT ADP DET,en,True,5,0
20937740,"council_NOUN of_ADP Berne_NOUN ,_. and_CONJ",67,"council of Berne , and",NOUN ADP NOUN PUNCT CONJ,"council of Berne , and",0_16_ORG,"council of berne , and",NOUN ADP NOUN PUNCTCONJ,en,True,5,0
1224818,"Compounds_NOUN ,_. ""_. Wiley_NOUN ,_.",108,"Compounds , "" Wiley ,",NOUN PUNCT PUNCT NOUN PUNCT,"Compounds , "" Wiley ,",0_9_ORG 14_19_PERSON,"compound , "" wiley ,",NOUN PUNCT PUNCT NOUN PUNCT,en,True,5,0
16841853,"control_NOUN technologies_NOUN (_. e.g._X ,_.",42,"control technologies ( e.g. ,",NOUN NOUN PUNCT X PUNCT,"control technologies ( e.g. ,",0_20_ORG,"control technologies ( e.g. ,",NOUN NOUN PUNCT ADV PUNCT,en,True,5,0
11197216,conferred_VERB on_ADP the_DET Parliament_NOUN ...,67,conferred on the Parliament by,VERB ADP DET NOUN ADP,conferred on the Parliament by,17_27_ORG,confer on the parliament by,VERB ADP DET NOUN ADP,en,True,5,0
14120245,constitution of the Assembly of,62,constitution of the Assembly of,,constitution of the Assembly of,20_28_ORG,constitution of the assembly of,NOUN ADP DET NOUN ADP,en,True,5,0
4908824,"come_VERB into_ADP the_DET Treasury_NOUN ,_.",132,"come into the Treasury ,",VERB ADP DET NOUN PUNCT,"come into the Treasury ,",14_22_ORG,"come into the treasury ,",VERB ADP DET NOUN PUNCT,en,True,5,0
1473913,Congress and the President 's,599,Congress and the President 's,,Congress and the President 's,0_8_ORG,congress and the president 's,NOUNCONJ DET NOUN PART,en,True,5,0
2245428,"Corps of Engineers , Waterways",1674,"Corps of Engineers , Waterways",,"Corps of Engineers , Waterways",0_18_ORG,"corps of engineers , waterways",NOUN ADP NOUN PUNCT NOUN,en,True,5,0
1458524,"Congress , and to show",123,"Congress , and to show",,"Congress , and to show",0_8_ORG,"congress , and to show",NOUN PUNCTCONJ PART VERB,en,True,5,0


In [ ]:
index_df

In [ ]:
    index_df.loc[index_df.pos_sent.str.contains(n1),'comp_class']=1
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n2),'comp_class']=2
    index_df.loc[index_df.pos_sent.str.contains(n3),'comp_class']=3
    index_df.loc[index_df.pos_sent.str.contains(n4),'comp_class']=4
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n5),'comp_class']=5